In [ ]:
#using Markdown; Markdown.parse_file("README.md")

In [ ]:
MODEL_PATH_KEY = :hPC;
include("heavyloadings.jl");

In [ ]:
half_b = load(MODEL_FOLDER*"\\bunny\\half_bunny_unmod.ply");
half_b_rot = load(MODEL_FOLDER*"\\bunny\\half_bunny_rot1.ply");

In [ ]:
include("firsticp.jl")

In [ ]:
IJuliaCell(vis)

In [ ]:
placeMeshInLife(vis,half_b,"hbunny","darkred");
placeMeshInLife(vis,half_b_rot,"rotbunny","olivedrab2")

In [ ]:
noisd_bun = noisifyMesh(half_b,wfactor=0.5);

In [ ]:
noisd_bun_10 = noisifyMesh(half_b,wfactor=1.5);

In [ ]:
placeMeshInLife(vis,noisd_bun,"n_bun","blue")

In [ ]:
placeMeshInLife(vis,noisd_bun_10,"n_bun10")

In [ ]:
#Workflow: (reading, reference)
#Input meshes/pointclouds
reference_pc = deepcopy(half_b);
reading_pc = noisifyMesh(half_b);
#Input's arrays
reference_array = vertices(reference_pc);
reading_array = vertices(reading_pc);
#KD tree for the reference array
kdtree_ref = KDTree(reference_array);
